Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in investment options input csv
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# eliminate repeat symbols
symbols = set(etfs)

In [7]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..........................................................................


In [8]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [9]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['SPY']

37.90375248590614

In [10]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [11]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['SPY']

0.1365054323891963

In [12]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.08307810382194525

In [13]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['SPY'])

Date
2015-01-31   176.85
2015-02-28   186.79
2015-03-31   183.85
2015-04-30   185.66
2015-05-31   188.05
2015-06-30   184.23
2015-07-31   188.39
2015-08-31   176.91
2015-09-30   172.39
2015-10-31   187.06
2015-11-30   187.74
2015-12-31   184.50
2016-01-31   175.31
2016-02-29   175.17
2016-03-31   186.95
2016-04-30   187.69
2016-05-31   190.88
2016-06-30   191.54
2016-07-31   198.53
2016-08-31   198.77
2016-09-30   198.78
2016-10-31   195.33
2016-11-30   202.53
2016-12-31   206.63
2017-01-31   210.33
2017-02-28   218.59
2017-03-31   218.87
2017-04-30   221.04
2017-05-31   224.16
2017-06-30   225.59
2017-07-31   230.23
2017-08-31   230.90
2017-09-30   235.55
2017-10-31   241.10
2017-11-30   248.47
2017-12-31   251.48
2018-01-31   265.66
2018-02-28   256.00
2018-03-31   248.98
2018-04-30   250.27
2018-05-31   256.35
2018-06-30   257.82
2018-07-31   267.38
2018-08-31   275.91
2018-09-30   277.55
2018-10-31   258.37
2018-11-30   263.16
2018-12-31   239.99
2019-01-31   259.21
2019-02-28   26

In [14]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['SPY']

Date
2015-01-31     nan
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2021-02-28    0.03
2021-03-31    0.05
2021-04-30    0.05
2021-05-31    0.01
2021-06-30    0.01
Freq: M, Name: SPY, Length: 78, dtype: float64

In [15]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['SPY']

0.18150198160345066

In [16]:
# read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# for each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = []
for line in lines:
    # copy empty and comment lines
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # split line by comma delimiter; strip the fields
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # copy header
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')